# Association Rules: Market Basket Analysis Instacart

* **Practical Business Python**: https://pbpython.com/market-basket-analysis.html
* **Market Basket Analysis Notebook**: https://github.com/chris1610/pbpython/blob/master/notebooks/Market_Basket_Intro.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import time

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.model_selection import train_test_split
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
products = pd.read_csv('../../data/01_raw/instacart_2017_05_01/products.csv')
aisles = pd.read_csv('../../data/01_raw/instacart_2017_05_01/aisles.csv')
departments = pd.read_csv('../../data/01_raw/instacart_2017_05_01/departments.csv')

order_products__prior = pd.read_csv('../../data/01_raw/instacart_2017_05_01/order_products__prior.csv')

In [ ]:
print('Products Row & Column Count: ', products.shape)
print('Aisles Row & Column Count: ', aisles.shape)
print('Department Row & Column Count: ', departments.shape)
print('Order Row & Column Count: ', order_products__prior.shape)

### Combine Datasets

We need to make sure that we can match carts with the names of products as well as users. In order to do this we will need to combine the dataset on a common key. Let's first start by combining product and ailes dataset.

In [43]:
prod_ailes = products.merge(aisles, 
              how='outer', 
              on='aisle_id', 
               suffixes=('_x', '_y')
              )

In [44]:
product_dataset = prod_ailes.merge(departments, 
                how='outer', 
                on='department_id')

In [45]:
specific_orders = order_products__prior.merge(product_dataset, 
                how='left', 
                on='product_id')

In [46]:
baskets = specific_orders.merge(order_test, 
                     how='left', 
                     on='order_id')

In [7]:
# baskets_samp = baskets.head(3000)

There are some interesting columns that we don't need
1. add to cart order
1. aile_id
1. depaertment_id
1. aisle
1. eval_set
1. order_number
1. order_dow
1. hour of the day
 

In [48]:
baskets.drop(columns = ['add_to_cart_order', 'reordered', 'aisle_id', 
                        'department_id', 'aisle', 'eval_set', 'order_number', 
                        'order_dow', 'order_hour_of_day'], inplace=True)

In [50]:
baskets.shape

(32434489, 6)

In [52]:
baskets = baskets.loc[(baskets['department']!='personal care')|
                                      (baskets['department']!='household')|
                                      (baskets['department']!='babies')|
                                      (baskets['department']!='pets')|
                                      (baskets['department']!='other')|
                                     (baskets['department']!='alcohol')]

In [53]:
baskets.drop(columns=['product_id', 'days_since_prior_order', 'department'], inplace=True)

In [54]:
baskets.head()

,order_id,product_name,user_id
0,2,Organic Egg Whites,202279
1,2,Michigan Organic Kale,202279
2,2,Garlic Powder,202279
3,2,Coconut Butter,202279
4,2,Natural Sweetener,202279


In [56]:
baskets['product_count'] = 1

In [58]:
# baskets.to_csv('../../data/02_intermediate/baskets_spark.csv')

## Make our Association Rules

In [15]:
df = baskets.copy()

In [16]:
df.columns

Index(['order_id', 'product_name', 'product_count'], dtype='object')

In [17]:
df['product_name'] = df['product_name'].str.strip()

In [21]:
df['order_id'] = df['order_id'].astype('str')

In [22]:
df

,order_id,product_name,product_count
0,2,Organic Egg Whites,1
1,2,Michigan Organic Kale,1
2,2,Garlic Powder,1
3,2,Coconut Butter,1
4,2,Natural Sweetener,1
...,...,...,...
2995,322,Yellow Onions,1
2996,322,French Green Beans,1
2997,322,Artichokes,1
2998,323,Organic Hass Avocado,1


In [23]:
basket = (df.groupby(['order_id', 'product_name'])['product_count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))

In [25]:
basket.head()

product_name,0% Greek Strained Yogurt,1 Apple + 1 Pear Fruit Bar,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Apple Juice Original,100% Cranberry Juice,100% Guava Juice,100% Juice No Added Sugar Orange Tangerine,100% Juice No Sugar Added Apple,100% Lactose Free Fat Free Milk,100% Natural Spring Water Minis,100% Pineapple Juice,100% Pure Apple Juice,100% Pure Rosemary,100% Raw Coconut Water,100% Recycled Bath Tissue Rolls,100% Recycled Bathroom Tissue,100% Recycled Paper Towels,100% Spring Water,100% Whole Wheat Bread,100% Whole Wheat Hot Dog Buns,13 Gallon Kitchen Drawstring Trash Bags,13 Gallon Tall Kitchen Bags,13 Gallon Tall Kitchen Drawstring Bags,2% Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,2-Ply 100% Recycled Bathroom Tissue,24/7 Performance Cat Litter,3 Minute Miracle Moist Deep Conditioning Treatment,3 lb Clementines,34% Cacao Milk Chocolate Bar,37% Less Sodium Soy Sauce,6 Cheese Italian Shredded Cheese,6 OZ LA PANZANELLA CROSTINI ORIGINAL CRACKERS,66 Calorie Unfiltered Ginger Ale,7 Grain Waffles,85% Lean Ground Beef,90 Calories Low Fat Chocolate Chunk Granola Bars,Acorn Squash,Advance White Fluoride Anticavity Tartar Control Extreme Whitening Fresh Mint Flavor Toothpaste,Aged White Cheddar Baked Rice And Corn Puffs,Aged White Cheddar Gluten-Free Baked Rice And Corn Puffs,Air Chilled Organic Boneless Skinless Chicken Breasts,All Natural Apricot Sparkling Water,All Natural Boneless Skinless Chicken Breasts,All Natural Gluten Free Probiotic Pineapple Yoghurt,All Natural Gluten Free Teff Wraps,All Natural Honey Almond Butter,All Natural Marinara Pasta Sauce,All Natural Marinara Sauce,All Natural No Stir Creamy Almond Butter,All Natural Powder Cleanser,All Natural Roasted Garlic Hummus,All Natural Seasoned Waffle Cut Fries,All Natural Virgin Lemonade,All Purpose Cleaner,All Whites 100% Egg Whites,Almond & Apricot Bar,Almond Breeze Original Almond Milk,Almond Cheddar Style Shredded Cheese Alternative,Almond Flour Tortillas,Almond Meal/Flour,...,Wheat Thins Reduced Fat Crackers,White Cheddar Macaroni & Cheese Cups,White Cheddar Popcorn,White Cheddar Semisoft Cheese,White Chocolate Raspberry Truffle Ice Cream,White Cooking Wine,White Corn,White Corn Tortilla Chips,White Distilled Vinegar,White Giant Paper Towel Rolls,White Horseradish,White Sandwich Bread,White Tea Peach Mango Tea Bags,Whole Almonds,Whole Grain Cheddar Baked Snack Crackers,Whole Grain Oatmeal Bread,"Whole Grain Organic Waffles, Totally Original",Whole Grain Pita Bread,Whole Grain White Corn Salted Tortilla Chips,Whole Grains Health Nut Bread,Whole Hearts of Palm,Whole Milk,Whole Milk Plain Yogurt,Whole Milk With Vitamin D,Whole Organic Omega 3 Milk,Whole Strawberries,Whole Vitamin D Milk,Whole Wheat Bunnies Baked Snack Crackers,Whole Wheat Sourdough,Whole White Mushrooms,Wild Arugula,Wild Berry Low Fat Clusters & Flakes,Wild Berry Smoothie,Wild Blend Rice,Wild Skipjack Light Tuna,Wild Sockeye Salmon,Wild Sweet Orange Herbal Tea,Wint-O-Green,With Bleach Cleanser,XL Emerald White Seedless Grapes,XX Espresso Iced Coffee With Almond Milk,Yellow Bell Pepper,Yellow Grape Tomatoes,Yellow Onions,Yellow Potato,Yellow Straightneck Squash,Yo Baby Organic Whole Milk Banana Mango Yogurt,YoKids Blueberry & Strawberry/Vanilla Yogurt,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry",YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,"Yogurt, Lowfat, Strawberry","Yogurt, Nonfat, Organic, Plain",Yuba Tofu Skin,Yukon Gold Potatoes 5lb Bag,ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Cream Soda,Zero Calories Berry Nutrient Enhanced Water,Zinfandel,Zucchini Noodles,gel hand wash sea minerals,with Crispy Almonds Cereal
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [26]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = basket.applymap(encode_units)

In [27]:
basket_sets.head()

product_name,0% Greek Strained Yogurt,1 Apple + 1 Pear Fruit Bar,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Apple Juice Original,100% Cranberry Juice,100% Guava Juice,100% Juice No Added Sugar Orange Tangerine,100% Juice No Sugar Added Apple,100% Lactose Free Fat Free Milk,100% Natural Spring Water Minis,100% Pineapple Juice,100% Pure Apple Juice,100% Pure Rosemary,100% Raw Coconut Water,100% Recycled Bath Tissue Rolls,100% Recycled Bathroom Tissue,100% Recycled Paper Towels,100% Spring Water,100% Whole Wheat Bread,100% Whole Wheat Hot Dog Buns,13 Gallon Kitchen Drawstring Trash Bags,13 Gallon Tall Kitchen Bags,13 Gallon Tall Kitchen Drawstring Bags,2% Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,2-Ply 100% Recycled Bathroom Tissue,24/7 Performance Cat Litter,3 Minute Miracle Moist Deep Conditioning Treatment,3 lb Clementines,34% Cacao Milk Chocolate Bar,37% Less Sodium Soy Sauce,6 Cheese Italian Shredded Cheese,6 OZ LA PANZANELLA CROSTINI ORIGINAL CRACKERS,66 Calorie Unfiltered Ginger Ale,7 Grain Waffles,85% Lean Ground Beef,90 Calories Low Fat Chocolate Chunk Granola Bars,Acorn Squash,Advance White Fluoride Anticavity Tartar Control Extreme Whitening Fresh Mint Flavor Toothpaste,Aged White Cheddar Baked Rice And Corn Puffs,Aged White Cheddar Gluten-Free Baked Rice And Corn Puffs,Air Chilled Organic Boneless Skinless Chicken Breasts,All Natural Apricot Sparkling Water,All Natural Boneless Skinless Chicken Breasts,All Natural Gluten Free Probiotic Pineapple Yoghurt,All Natural Gluten Free Teff Wraps,All Natural Honey Almond Butter,All Natural Marinara Pasta Sauce,All Natural Marinara Sauce,All Natural No Stir Creamy Almond Butter,All Natural Powder Cleanser,All Natural Roasted Garlic Hummus,All Natural Seasoned Waffle Cut Fries,All Natural Virgin Lemonade,All Purpose Cleaner,All Whites 100% Egg Whites,Almond & Apricot Bar,Almond Breeze Original Almond Milk,Almond Cheddar Style Shredded Cheese Alternative,Almond Flour Tortillas,Almond Meal/Flour,...,Wheat Thins Reduced Fat Crackers,White Cheddar Macaroni & Cheese Cups,White Cheddar Popcorn,White Cheddar Semisoft Cheese,White Chocolate Raspberry Truffle Ice Cream,White Cooking Wine,White Corn,White Corn Tortilla Chips,White Distilled Vinegar,White Giant Paper Towel Rolls,White Horseradish,White Sandwich Bread,White Tea Peach Mango Tea Bags,Whole Almonds,Whole Grain Cheddar Baked Snack Crackers,Whole Grain Oatmeal Bread,"Whole Grain Organic Waffles, Totally Original",Whole Grain Pita Bread,Whole Grain White Corn Salted Tortilla Chips,Whole Grains Health Nut Bread,Whole Hearts of Palm,Whole Milk,Whole Milk Plain Yogurt,Whole Milk With Vitamin D,Whole Organic Omega 3 Milk,Whole Strawberries,Whole Vitamin D Milk,Whole Wheat Bunnies Baked Snack Crackers,Whole Wheat Sourdough,Whole White Mushrooms,Wild Arugula,Wild Berry Low Fat Clusters & Flakes,Wild Berry Smoothie,Wild Blend Rice,Wild Skipjack Light Tuna,Wild Sockeye Salmon,Wild Sweet Orange Herbal Tea,Wint-O-Green,With Bleach Cleanser,XL Emerald White Seedless Grapes,XX Espresso Iced Coffee With Almond Milk,Yellow Bell Pepper,Yellow Grape Tomatoes,Yellow Onions,Yellow Potato,Yellow Straightneck Squash,Yo Baby Organic Whole Milk Banana Mango Yogurt,YoKids Blueberry & Strawberry/Vanilla Yogurt,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry",YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,"Yogurt, Lowfat, Strawberry","Yogurt, Nonfat, Organic, Plain",Yuba Tofu Skin,Yukon Gold Potatoes 5lb Bag,ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Cream Soda,Zero Calories Berry Nutrient Enhanced Water,Zinfandel,Zucchini Noodles,gel hand wash sea minerals,with Crispy Almonds Cereal
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)

In [42]:
frequent_itemsets.head()

,support,itemsets
0,0.022727,(2% Reduced Fat Milk)
1,0.116883,(Bag of Organic Bananas)
2,0.129870,(Banana)
3,0.025974,(Blueberries)
4,0.032468,(Carrots)


In [41]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Raspberries),(Bag of Organic Bananas),0.038961,0.116883,0.022727,0.583333,4.990741,0.018173,2.119481
1,(Bag of Organic Bananas),(Organic Raspberries),0.116883,0.038961,0.022727,0.194444,4.990741,0.018173,1.193014
2,(Banana),(Organic Baby Spinach),0.129870,0.081169,0.022727,0.175000,2.156000,0.012186,1.113735
3,(Organic Baby Spinach),(Banana),0.081169,0.129870,0.022727,0.280000,2.156000,0.012186,1.208514
